In [ ]:
!date

In [ ]:
pwd

In [ ]:
whos

In [ ]:
import os
import numpy as np
from tqdm import tqdm

In [ ]:
def get_random_string():
    import string
    import random
    return ''.join([random.choice(string.ascii_letters) for _ in range(10)])

def safe_mkdir(dirpath):
    if not os.path.exists(dirpath):
        os.mkdir(dirpath)

In [ ]:
analysis_name = 'CVAE_COMORB_D4B1_'+get_random_string()
print(analysis_name)

In [ ]:
safe_mkdir(f'../tf_weights/{analysis_name}')

In [ ]:
save_dir_stage1 = f'../tf_weights/{analysis_name}/stage_1'
save_dir_stage2 = f'../tf_weights/{analysis_name}/stage_2'

In [ ]:
%%time
for iter in tqdm(range(100)):
    os.system(f'python initialize_CVAE-COMORB.py {save_dir_stage1} {iter} 10 None')

2023-12-13 13:33:24.754986: W tensorflow/stream_executor/gpu/asm_compiler.cc:81] Running ptxas --version returned 256
2023-12-13 13:33:24.787836: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] Internal: ptxas exited with non-zero error code 256, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2023-12-13 13:33:24.878555: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcublas.so.10
/data/aglinska/anaconda3/envs/tf231/lib/python3.8/site-packages/scipy/__init__.py:143: UserWarning: A NumPy version >=1.19.5 and <1.27.0 is required for this version of SciPy (detected version 1.18.5)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
 12%|█▏        | 6/50 [30:28<3:24:45, 279.21s/it]2023-12-13 13:39:20.177720: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart

CPU times: user 751 ms, sys: 594 ms, total: 1.34 s
Wall time: 4h 33min 20s


In [ ]:
files_varexp = [file for file in os.listdir(save_dir_stage1) if file.startswith('varexp_attmp_')];files_varexp.sort()

varexps = np.array([np.load(os.path.join(save_dir_stage1,f))[-1] for f in files_varexp])

file_nums = np.array([int(f.replace('varexp_attmp_','').replace('.npy','')) for f in files_varexp])
ORD = np.argsort(varexps)[-1::-1]
[print(f'{files_varexp[o]} {varexps[o]:.2f}') for o in ORD[0:20]];
IDs_best = file_nums[ORD[0:20]]
stage_2_weights = [os.path.join(save_dir_stage1,f'CVAE_init_attmp_{val}') for val in IDs_best]

In [ ]:
%%time
for iter in tqdm(range(20)):
    os.system(f'python initialize_CVAE-COMORB.py {save_dir_stage2} {iter} 100 {stage_2_weights[iter]}')

100%|██████████| 20/20 [19:43:30<00:00, 3550.54s/it]

CPU times: user 2.12 s, sys: 2.09 s, total: 4.22 s
Wall time: 19h 43min 30s
